In [1]:
import openmatrix as omx
import pandas as pd
import numpy as np
import yaml
from utility import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
summary_dir = params['summary_dir']

time_periods = params['periods']
perf_measure_columns = params['final_columns']

best_path_skim_extension = params['best_path_skim_extension']

annual_transit_factor = params['annual_transit_factor']
annual_auto_factor = params['annual_auto_factor']

filename_extension = params['filename_extension']

In [4]:
print(perf_measure_columns)

['Concept_ID', 'Metric_ID', 'Metric_name', 'Submetric', 'Description', 'Population', 'Period', 'Geography', 'Zone_ID', 'Origin_zone', 'Dest_zone', 'Purpose', 'Mode', 'Income', 'Value', 'Units', 'Total_Increment']


## Final Code

In [5]:
tod_skims = pd.read_parquet(_join(preprocess_dir, 'tod_min_perceived_travel_time.parquet'))
tod_skims = tod_skims.reset_index()
tod_skims.head(3)

tp,orig,dest,am,ea,ev,md,pm
0,1,1,NaN,NaN,NaN,NaN,NaN
1,1,2,9361.0,7383.0,7338.0,7702.0,9353.0
2,1,3,8118.0,7171.0,7132.0,7299.0,7183.0


In [6]:
# tod_skims.describe()

In [7]:
time_periods

['am', 'md', 'pm', 'ev', 'ea']

In [8]:
all_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster.parquet'))
#all_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster_2050_Baseline_R2_Run4.parquet'))
all_trips = all_trips.loc[all_trips['trip_mode'].isin([6,7,8])]
all_trips.head(0)

,hh_id,person_id,inbound,tour_purpose,orig_purpose,dest_purpose,orig_taz,dest_taz,depart_hour,trip_mode,sampleRate,trip_type,trips,transbay_od,orig_rdm_zones,orig_super_dist,orig_county,dest_rdm_zones,dest_super_dist,dest_county,home_zone,income,Income,pp_share,link21_trip_purp,Period,Mode


In [9]:
connected_ods = {}
final_trips = []
for period in time_periods:
    tp_df = all_trips[all_trips['Period'] == period.lower()]
    tod_skims_df = tod_skims[['orig', 'dest', period.lower()]].copy()
    tod_skims_df = tod_skims_df.dropna()
    print(f'Connected ODs in {period}: {len(tod_skims_df):,}')
    connected_ods[period] = len(tod_skims_df)
    tod_skims_df.columns = ['orig_taz', 'dest_taz', 'perc_tt']
    tp_df = pd.merge(tp_df, tod_skims_df, on = ['orig_taz', 'dest_taz'], how='inner')
    final_trips.append(tp_df)

Connected ODs in am: 10,857,718
Connected ODs in md: 10,778,586
Connected ODs in pm: 10,797,153
Connected ODs in ev: 10,769,171
Connected ODs in ea: 10,875,877


In [10]:
df_connected_ods = pd.DataFrame({'Period': connected_ods.keys(), 'Connected_ODs': connected_ods.values()})
df_connected_ods.to_csv(_join(summary_dir,
                              'A1.1'+ '_Connected_ODs_' + concept_id + '_region' + filename_extension + '.csv'),
                        index=None)

In [11]:
final_trips = pd.concat(final_trips)
len(final_trips)

2757724

In [12]:
final_trips[1:3]

,hh_id,person_id,inbound,tour_purpose,orig_purpose,dest_purpose,orig_taz,dest_taz,depart_hour,trip_mode,sampleRate,trip_type,trips,transbay_od,orig_rdm_zones,orig_super_dist,orig_county,dest_rdm_zones,dest_super_dist,dest_county,home_zone,income,Income,pp_share,link21_trip_purp,Period,Mode,perc_tt
1,2226843,5201246.0,0,work_med,Home,work,273,452,7,6,1.0,INM,1.0,0.0,San Francisco_37,3,1,San Francisco_16,3,1,273,44915,30k_to_60k,100.0,work,am,WALK_TRANSIT,3645.0
2,2226745,5201040.0,0,social,Home,social,273,452,9,6,1.0,INM,1.0,0.0,San Francisco_37,3,1,San Francisco_16,3,1,273,0,<30k,100.0,social,am,WALK_TRANSIT,3645.0


In [13]:
# Correct for the fact that perceived travel time column has been magnified by a factor of 100 (its unit is minutes). 
final_trips['perc_tt'] = final_trips['perc_tt']/100

In [14]:
summary_cols = params['description_a1.1']
summary_cols

{'perc_tt': ['perceived total travel time',
  'A1.1',
  'minutes',
  'perceived travel travel time']}

In [15]:
pp_df = []
reg_df = []
county_df = []
sd_df = []
tb_df = []
rdm_df = []
pp_df_rdm = []

for columns in summary_cols:

    #for period in time_periods:
    #print(period)

    df_temp = final_trips# [final_trips['Period'] == period.lower()]

    df_temp[columns+'_trips'] = df_temp[columns] * df_temp['trips']

    #regional value
    reg_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column=columns+'_trips')
    reg_df_temp1 = reg_df_temp1.rename(columns={'Value': columns+'_trips'})
    
    reg_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column='trips')
    reg_df_temp2 = reg_df_temp2.rename(columns={'Value': 'trips'})
    reg_df_temp = pd.merge(reg_df_temp1, reg_df_temp2, on=['Period', 'Income'], how='left')
    reg_df_temp[columns] = reg_df_temp[columns+'_trips']/reg_df_temp['trips']
    reg_df_temp = reg_df_temp[['Period', 'Income', columns]]
    reg_df_temp = reg_df_temp.rename(columns={columns:'Value'})
    reg_df_temp['Population'] = 'Whole Population'
    reg_df_temp['Origin_zone'] = ''
    reg_df_temp['Dest_zone'] = ''
    reg_df_temp['Zone_ID'] = ''
    reg_df_temp['Concept_ID'] = concept_id
    reg_df_temp['Geography'] = 'Regional'
    reg_df_temp['Purpose'] = ''
    reg_df_temp['Mode'] = ''
    reg_df_temp['Total_Increment'] = ''
    reg_df_temp['Metric_ID'] = summary_cols[columns][1]
    reg_df_temp['Submetric'] = summary_cols[columns][1]+'.1'
    reg_df_temp['Description'] = 'Average ' + summary_cols[columns][0]+ ' in the region'
    reg_df_temp['Units'] = summary_cols[columns][2]
    reg_df_temp['Metric_name'] = summary_cols[columns][3]
    reg_df.append(reg_df_temp)
        
reg_df = pd.concat(reg_df)

In [17]:
all_dfs = [reg_df]

for dfs in all_dfs:
    metric_name = '_perceived_travel_time_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0], dfs['Value'].sum())

30 A1.1.1 perceived travel travel time 2122.0915334130023
30 A1.1.2 perceived travel travel time 2414.824930505644
2568 A1.1.3 perceived travel travel time 407821.5027786014
686790 A1.1.4 perceived travel travel time 66258460.95393775
24938 A1.1.5 perceived travel travel time 3445815.646804338
30 A1.1.6 perceived travel travel time 22.464716171402543
686790 A1.1.7 perceived travel travel time 298942.13050205196
